In [1]:
pip install langid

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd   
import os 
import numpy as np
import string
import re 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_val_predict
nltk.download('stopwords')
from translate import Translator
import langid
from sklearn.model_selection import train_test_split
import pickle



translator= Translator(to_lang="English")

from sklearn import set_config
set_config(display='diagram')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harisnaveed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv('archive/reviews.csv', encoding='latin-1')
data 

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0
...,...,...,...,...,...,...,...,...
67981,B081H6STQQ,jande,5,"August 16, 2019",False,"Awesome Phone, but finger scanner is a big mis...",I love the camera on this phone. The screen is...,1.0
67982,B081H6STQQ,2cool4u,5,"September 14, 2019",False,Simply Amazing!,I've been an Xperia user for several years and...,1.0
67983,B081H6STQQ,simon,5,"July 14, 2019",False,"great phon3, but many bugs need to fix. still ...",buy one more for my cousin,NaN
67984,B081TJFVCJ,Tobiasz Jedrysiak,5,"December 24, 2019",True,Phone is like new,Product looks and works like new. Very much re...,NaN


In [4]:
new_dataframe = pd.DataFrame(columns=['text','score']) 
new_dataframe['score'] = data['rating']
new_dataframe['text'] = data['body']

In [5]:
new_dataframe

,text,score
0,I had the Samsung A600 for awhile which is abs...,3
1,Due to a software issue between Nokia and Spri...,1
2,"This is a great, reliable phone. I also purcha...",5
3,"I love the phone and all, because I really did...",3
4,The phone has been great for every purpose it ...,4
...,...,...
67981,I love the camera on this phone. The screen is...,5
67982,I've been an Xperia user for several years and...,5
67983,buy one more for my cousin,5
67984,Product looks and works like new. Very much re...,5


In [6]:
new_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67986 entries, 0 to 67985
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    67965 non-null  object
 1   score   67986 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [7]:
new_dataframe = new_dataframe.dropna(how='any',axis=0)

In [8]:
good = new_dataframe['score'] >= 3
bad = new_dataframe['score'] < 3

In [9]:
new_dataframe.loc[good,'score'] = 1
new_dataframe.loc[bad, 'score'] = 0

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
new_dataframe

,text,score
0,I had the Samsung A600 for awhile which is abs...,1
1,Due to a software issue between Nokia and Spri...,0
2,"This is a great, reliable phone. I also purcha...",1
3,"I love the phone and all, because I really did...",1
4,The phone has been great for every purpose it ...,1
...,...,...
67981,I love the camera on this phone. The screen is...,1
67982,I've been an Xperia user for several years and...,1
67983,buy one more for my cousin,1
67984,Product looks and works like new. Very much re...,1


In [11]:
#new_dataframe['text'] = [str(text) for text in new_dataframe['text']]

In [12]:
def split_labels(data, label_feature):
    """
    Split the given column of of the data, returning the full data set (without that
    feature) and the split off feature.
    """
    return data.drop(columns=label_feature), data[label_feature]

In [13]:
# Splits the data into the train and test set by stratifiying on the label.
train_set, test_set = train_test_split(new_dataframe, stratify=new_dataframe['score'], test_size=0.2, random_state=420)

In [14]:
all_stop_words = set(stopwords.words('english'))
all_stop_words.remove('not')
ps = PorterStemmer()

In [15]:
def BagOfWords(X):
    """Takes each entry in the dataset and lowers the character. 
    It removes any symbols/punctuation. 
    It then applys the portStemmer to anything that is not a stopword and adds it to the list corpus."""
    corpus = []
    for text in X['text']:
        review = text.lower()
        review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
        review = review.split()
        review = [word for word in review if word not in all_stop_words]
        review = " ".join(ps.stem(word) for word in review)
        corpus.append(review)
    return corpus
    

In [16]:
corpus = BagOfWords(train_set)

In [17]:
corpus

['best phone ever use great batteri life beauti screen love plu thing keep fragil bodi beauti glass rest everyth deluc',
 'good produc',
 'work expect factori unlock thing plug not america come adapt solv littl issu',
 'un excelent tel fono un precio acces su configuraci n de memoria procesador se complementan para un buen desemp',
 'overal realli great phone inher flaw signal recept compar poor actual carrier network one would assum motorola famou radio would offer superior product six year old samsung reliabl receiv call text messag moto x not issu forc return phone otherwis pretti fantast',
 'disappoint phone model longer sold due irrepar softwar issu appl would give warranti purchas wast time without phone twice two thumb one',
 'absolut love phone one peopl obsess gadget phone big one mobil subscrib current use galaxi well iphon use note love mobil let upgrad better camera case use note previous bit much x e issu might voic recognit excel respons like phone calib mani shortcut tri

In [18]:
X, y = split_labels(train_set, 'score')

In [29]:
all_stop_words = stopwords.words('english')
all_stop_words.remove('not')
ps = PorterStemmer()

def text_editer(text):
    """Takes the text and lowers the characters.. 
    It removes any symbols/punctuation. 
    It then applys the portStemmer to anything that is not a stopword and returns the review.."""
    review = text.lower()
    review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
    review = review.split()
    review = [word for word in review if word not in set(all_stop_words)]
    review = " ".join(ps.stem(word) for word in review)
    return review

class EditingText(BaseEstimator, TransformerMixin):
    """Transforms the data using the text editor function and returns it as a numpy array."""
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        data = np.array([text_editer(text) for text in X['text']]).flatten()
        return data

In [30]:
# Preprocesses the data by using the editing text class, vecotrizing, and then scaling the data.
preprocessor = Pipeline(steps=[
    ('EditingText', EditingText()), 
    ('TfidfVectorizer', TfidfVectorizer()),
    ('TfidfVectorizerTransformer', TfidfTransformer()),
    ('Simple_Imputer', SimpleImputer()),
    ('StandardScaler', StandardScaler(with_mean=False)),
])

In [31]:
preprocessor.fit(X,y)

Pipeline(steps=[('EditingText', EditingText()),
                ('TfidfVectorizer', TfidfVectorizer()),
                ('TfidfVectorizerTransformer', TfidfTransformer()),
                ('Simple_Imputer', SimpleImputer()),
                ('StandardScaler', StandardScaler(with_mean=False))])

In [32]:
transformed = preprocessor.transform(X)

In [33]:
print(transformed)

  (0, 69)	1.1826415519751274
  (0, 214)	1.9127927860074758
  (0, 598)	1.0856434647914206
  (0, 628)	1.674527244519814
  (0, 987)	74.18353411157676
  (0, 1369)	5.201331407719405
  (0, 1440)	0.7662314870379207
  (0, 1514)	4.29951847162648
  (0, 1586)	0.350681774928786
  (0, 1819)	2.6994845638834297
  (0, 1855)	1.6157556614410649
  (0, 2141)	0.6241782108321581
  (0, 2193)	2.983529060781926
  (0, 2234)	2.9866170224956052
  (0, 2423)	1.6582555779804986
  (0, 2689)	2.4729869131601903
  (0, 2721)	13.388427598077667
  (0, 2888)	0.5550812967166282
  (0, 2906)	1.4653039577106595
  (0, 3117)	7.107454864475932
  (0, 3306)	0.9462382217080458
  (0, 3374)	2.021423736756585
  (0, 3383)	8.510463752999737
  (0, 3619)	3.013728231785331
  (0, 3729)	260.70289606123225
  :	:
  (67961, 19165)	3.9751046200283437
  (67961, 19746)	5.388115043792827
  (67961, 20282)	11.797818647110201
  (67961, 20316)	26.138391283849128
  (67961, 20339)	4.0855265114669885
  (67961, 20342)	2.0934505695432137
  (67962, 2427)	3.385

In [34]:
# Function that prints out the accuracy, precision, and recall scores. 
def metric_score(y,y_pred):
    metric_dict = {'Accuracy': accuracy_score, 'Precision': precision_score, 'Recall': recall_score}
    for metric, func in metric_dict.items():
        print(f'{metric}: {func(y, y_pred)}')

# Random Forest Classifer #

In [35]:
X,y = split_labels(new_dataframe, 'score')

In [36]:
#Creates a pipeline with the preprocessor and the RFC, then fits it.
pipelineRF = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('RandomForest', RandomForestClassifier()),
])
pipelineRF.fit(X,y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('EditingText', EditingText()),
                                 ('TfidfVectorizer', TfidfVectorizer()),
                                 ('TfidfVectorizerTransformer',
                                  TfidfTransformer()),
                                 ('Simple_Imputer', SimpleImputer()),
                                 ('StandardScaler',
                                  StandardScaler(with_mean=False))])),
                ('RandomForest', RandomForestClassifier())])

In [37]:
y_pred = cross_val_predict(pipelineRF, X, y)

In [38]:
y_pred

array([1, 0, 1, ..., 1, 1, 1])

In [39]:
metric_score(y, y_pred)

Accuracy: 0.8812771279334952
Precision: 0.8931435117002128
Recall: 0.9572826129321432


# SGD Classifier # 

In [40]:
X,y = split_labels(new_dataframe, 'score')

In [41]:
#Creates a pipeline with the preprocessor and the SGDClassifier, then fits it.
pipelineSGD = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('SGD', SGDClassifier())
])
pipelineSGD.fit(X,y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('EditingText', EditingText()),
                                 ('TfidfVectorizer', TfidfVectorizer()),
                                 ('TfidfVectorizerTransformer',
                                  TfidfTransformer()),
                                 ('Simple_Imputer', SimpleImputer()),
                                 ('StandardScaler',
                                  StandardScaler(with_mean=False))])),
                ('SGD', SGDClassifier())])

In [42]:
y_pred = cross_val_predict(pipelineSGD, X, y)

In [43]:
metric_score(y, y_pred)

Accuracy: 0.8292650629000221
Precision: 0.8851600387972842
Recall: 0.8892310090813423


# Fine-Tune the System 
1. Fine-tune the hyperparameters using cross-validation. Treat your data 
transformation choices as hyperparameters, especially when you are not sure 
about them. Unless there are very few hyperparameter values to explore, prefer 
random search over grid search. If training is very long, you may prefer a Bayesian 
optimization approach. 
2. Try Ensemble methods. Combining your best models will often perform better 
than running them individually. 
3. Once you are confident about your final model, measure its performance on the 
test set to estimate the generalization error. 

In [ ]:
X,y = split_labels(new_dataframe, 'score')

In [ ]:
pipelineSGD.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'SGD', 'preprocessor__memory', 'preprocessor__steps', 'preprocessor__verbose', 'preprocessor__EditingText', 'preprocessor__TfidfVectorizer', 'preprocessor__TfidfVectorizerTransformer', 'preprocessor__Simple_Imputer', 'preprocessor__StandardScaler', 'preprocessor__TfidfVectorizer__analyzer', 'preprocessor__TfidfVectorizer__binary', 'preprocessor__TfidfVectorizer__decode_error', 'preprocessor__TfidfVectorizer__dtype', 'preprocessor__TfidfVectorizer__encoding', 'preprocessor__TfidfVectorizer__input', 'preprocessor__TfidfVectorizer__lowercase', 'preprocessor__TfidfVectorizer__max_df', 'preprocessor__TfidfVectorizer__max_features', 'preprocessor__TfidfVectorizer__min_df', 'preprocessor__TfidfVectorizer__ngram_range', 'preprocessor__TfidfVectorizer__norm', 'preprocessor__TfidfVectorizer__preprocessor', 'preprocessor__TfidfVectorizer__smooth_idf', 'preprocessor__TfidfVectorizer__stop_words', 'preprocessor__TfidfVectorizer__strip_accents

In [ ]:
param_grid = [
       {'preprocessor__StandardScaler__with_mean': [False],
         'preprocessor__TfidfVectorizer__use_idf' : [True, False],
         'preprocessor__TfidfVectorizer__norm': ['l2'],
         'preprocessor__TfidfVectorizer__strip_accents': ['ascii', 'unicode', None],
         'preprocessor__TfidfVectorizer__smooth_idf': [True, False],  
        }
         
]


search = GridSearchCV(pipelineSGD, param_grid, cv=2, scoring='accuracy', verbose=4)

In [ ]:
search.fit(X,y)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV 1/2] END preprocessor__StandardScaler__with_mean=False, preprocessor__TfidfVectorizer__norm=l2, preprocessor__TfidfVectorizer__smooth_idf=True, preprocessor__TfidfVectorizer__strip_accents=ascii, preprocessor__TfidfVectorizer__use_idf=True;, score=0.816 total time=  54.4s
[CV 2/2] END preprocessor__StandardScaler__with_mean=False, preprocessor__TfidfVectorizer__norm=l2, preprocessor__TfidfVectorizer__smooth_idf=True, preprocessor__TfidfVectorizer__strip_accents=ascii, preprocessor__TfidfVectorizer__use_idf=True;, score=0.807 total time=  49.2s
[CV 1/2] END preprocessor__StandardScaler__with_mean=False, preprocessor__TfidfVectorizer__norm=l2, preprocessor__TfidfVectorizer__smooth_idf=True, preprocessor__TfidfVectorizer__strip_accents=ascii, preprocessor__TfidfVectorizer__use_idf=False;, score=0.821 total time=  49.0s
[CV 2/2] END preprocessor__StandardScaler__with_mean=False, preprocessor__TfidfVectorizer__norm=l2, preproc

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('EditingText',
                                                         EditingText()),
                                                        ('TfidfVectorizer',
                                                         TfidfVectorizer()),
                                                        ('TfidfVectorizerTransformer',
                                                         TfidfTransformer()),
                                                        ('Simple_Imputer',
                                                         SimpleImputer()),
                                                        ('StandardScaler',
                                                         StandardScaler(with_mean=False))])),
                                       ('SGD', SGDClassifier())]),
             param_grid=[{'preprocessor__StandardScaler__with_mean': [False],
                          'preprocessor__TfidfVectorizer__norm': ['l2'],
                          'preprocessor__TfidfVectorizer__smooth_idf': [True,
                                                                        False],
                          'preprocessor__TfidfVectorizer__strip_accents': ['ascii',
                                                                           'unicode',
                                                                           None],
                          'preprocessor__TfidfVectorizer__use_idf': [True,
                                                                     False]}],
             scoring='accuracy', verbose=4)

# Using the Test Set on Model

In [ ]:
text, labels = split_labels(test_set, "score")

In [ ]:
text

In [ ]:
y_test_pred = search.predict(text)

In [ ]:
metric = accuracy_score(labels, y_test_pred)
metric

0.9384241889207681

# Saving Model

In [ ]:
with open('SGDmodel.pkl', 'wb') as f:
    pickle.dump(search, f, pickle.HIGHEST_PROTOCOL)